# ...in C

## Crossref API in C
by Cyrus Gomes and Vincent Scalfani


**Crossref API documentation:** https://api.crossref.org/swagger-ui/index.html

These recipe examples were tested on July 25, 2023.

*From our testing, we have found that the crossref metadata across publishers and even journals can vary considerably. As a result, it can be easier to work with one journal at a time when using the crossref API (e.g., particulary when trying to extract selected data from records).*

### Setup

First, install the CURL and jq package by typing the following command in the terminal:

In [ ]:
!sudo apt install curl jq libcurl4-openssl-dev

First we set a directory where we want the CrossRef directory for our projects to be created 

In [2]:
!mkdir CrossRef

We change the directory to the folder we created

In [ ]:
%cd CrossRef

## 1. Basic CrossRef API call

### Request data from crossref API

We initialize a folder for the current project that we are working on. And then change to that directory

In [4]:
!mkdir crossref_api_call

In [ ]:
%cd crossref_api_call

Then we utilize the following command (%%file) to create the following makefile which will compile our program and create an executable.

In [ ]:
%%file makefile


#sets the variable CC to gcc, which is used to build the program
CC=gcc

#enable debugging information and enable all compiler warnings
CFLAGS=-g -Wall

#sets the bin variable as the name of the binary file we are creating
BIN=crossref_api

#creates the binary file with the name we put
all: $(BIN)

#maps any file ending in .c to a binary executable. 
#"$<" represents the .c file and "$@" represents the target binary executable
%: %.c

#compiles the .c file using the gcc compiler with the CFLAGS and links 
#resulting binary with the CURL library
	$(CC) $(CFLAGS) $< -o $@ -lcurl

#clean target which removes specific files
clean:

#removes the binary file and an ".dSYM" (debug symbols for debugging) directories
#the RM command used -r to remove directories and -f to force delete
	$(RM) -rf $(BIN) *.dSYM


The command is used again to create our .c file which contains the code for the program

In [ ]:
%%file crossref_api.c

#include <curl/curl.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

/*CURL program that retrieves JSON data from the CrossRef API
This program allows custom request to be used along with the parameter*/


/* We are going to be inputting the custom email and doi like this: ./crossref_api -e "your_email@ua.edu" -d "10.1186/1758-2946-4-12"
If the arguments are missing then we use the default: "your_email@ua.edu" "10.1186/1758-2946-4-12"
*/

int main (int argc, char* argv[]){
    
    //if arguments are invalid just return
    if (argc > 5){                                                                                      
        printf("Error. Please try again correctly.\n");
        return -1;
    }

    //default parameter and request codes
    char parameter[100] = {};
    char request[500] = {}; 

    //if there is ./crossref_api -e/-d
    if ((argc == 1) || ((argc == 2) && ((strcmp(argv[1], "-e")==0) || (strcmp(argv[1], "-d")==0)))){
        //these arguments run the default parameters and keeps the codes as they are
        strcat(parameter,"your_email@ua.edu");
        strcat(request, "10.1186/1758-2946-4-12");
    }

    //if there is ./crossref_api -e "your_email@ua.edu"
    else if ((argc == 3) && (strcmp(argv[1], "-e")==0)){
        //only the parameter code is changed
        strcat(parameter,argv[2]);
        strcat(request, "10.1186/1758-2946-4-12");
    }

    //if there is ./crossref_api -e "your_email@ua.edu" -d
    else if ((argc == 4) && (strcmp(argv[1], "-e")==0) && (strcmp(argv[3], "-d")==0)){
        //only the parameter code is changed
        strcat(parameter,argv[2]);
        strcat(request, "10.1186/1758-2946-4-12");
    }

    //if there is ./crossref_api -e "your_email@ua.edu" -d "10.1186/1758-2946-4-12"
    else if ((argc == 5) && (strcmp(argv[1], "-e")==0) && (strcmp(argv[3], "-d")==0)){
        //both the parameter and request codes are changed
        strcat(parameter,argv[2]);
        strcat(request, argv[4]);
    }

    //if there is ./crossref_api -d "10.1186/1758-2946-4-12"
    else if ((argc == 3) && (strcmp(argv[1], "-d")==0)){
        //only the request code is changed
        strcat(parameter,"your_email@ua.edu");
        strcat(request, argv[2]);
    }

    //if there is ./crossref_api -d "10.1186/1758-2946-4-12" -e
    else if ((argc == 4) && (strcmp(argv[1], "-d")==0) && (strcmp(argv[3], "-e")==0)){
        //only the request code is changed
        strcat(parameter,"your_email@ua.edu");
        strcat(request, argv[2]);
    }

    //if there is ./crossref_api -d "10.1186/1758-2946-4-12" -e "your_email@ua.edu" 
    else if ((argc == 5) && (strcmp(argv[1], "-d")==0) && (strcmp(argv[3], "-e")==0)){
        //both the request and parameter codes are changed
        strcat(parameter,argv[4]);
        strcat(request, argv[2]);
    }

    else{
        printf("usage: ./crossref_api [-d] request [-e] parameter\n\n");
        printf("the api_req_par program is used to retrieve json data from the CrossRef API\n\n");
        printf("optional arguments\n");
        printf("\t -d doi          optional custom doi code; default is '10.1186/1758-2946-4-12'\n");
        printf("\t -e email        optional custom email; default is 'your_email@ua.edu'\n");
        return -1;
    }

    //initializes the CURL HTTP connection
    CURL *curl = curl_easy_init();

    //bits of the url that are joined together later
    char api[] = "https://api.crossref.org/works/";                                                                     
    char type1[] = "?mailto=";                        
    char url[2000];

    //checks if CURL initialization is a success or not
    if (!curl){                                                                                         
        fprintf(stderr, "init failed\n");
        return EXIT_FAILURE;
    }
        
    //combines all the bits to produce a functioning url
    sprintf(url, "%s%s%s%s", api, request, type1, parameter);                                             
                                          

    //sets the url to which the HTTP request will be sent to
    //first parameter is for the initialized curl HTTP request, second for the option to be set, and third for the value to be set
    curl_easy_setopt(curl, CURLOPT_URL, url);

    //if result is not retrieved then output error
    CURLcode result = curl_easy_perform(curl);

    //if result is not retrieved then output error
    if (result != CURLE_OK){                                                                            
        fprintf(stderr, "download problem: %s\n", curl_easy_strerror(result));
    }

    //deallocates memory for the CURL connection
    curl_easy_cleanup(curl);                                                                            
    return EXIT_SUCCESS;
}

The folowing program is run, and an executable is created after using the following command:

In [ ]:
!make

To output the data from the CrossRef API, we enter the following command:

In [10]:
!./crossref_api | jq '.'

{
  "status": "ok",
  "message-type": "work",
  "message-version": "1.0.0",
  "message": {
    "indexed": {
      "date-parts": [
        [
          2023,
          4,
          26
        ]
      ],
      "date-time": "2023-04-26T02:21:45Z",
      "timestamp": 1682475705288
    },
    "reference-count": 16,
    "publisher": "Springer Science and Business Media LLC",
    "issue": "1",
    "license": [
      {
        "start": {
          "date-parts": [
            [
              2012,
              7,
              6
            ]
          ],
          "date-time": "2012-07-06T00:00:00Z",
          "timestamp": 1341532800000
        },
        "content-version": "tdm",
        "delay-in-days": 0,
        "URL": "http://creativecommons.org/licenses/by/2.0"
      }
    ],
    "content-domain": {
      "domain": [],
      "crossmark-restriction": false
    },
    "short-container-title": [
      "J Cheminform"
    ],
    "published-print": {
      "date-parts": [
        [
          2

### Select Some Specific Data

In [11]:
!./crossref_api | jq '.["message"]["container-title"]'

[
  "Journal of Cheminformatics"
]


In [12]:
!./crossref_api | jq '.["message"]["title"]'

[
  "The Molecule Cloud - compact visualization of large collections of molecules"
]


In [13]:
!./crossref_api | jq '.["message"]["author"] | length'

2


We can add our email and the custom doi to run the API call

In [14]:
!./crossref_api -d "10.1186/1471-2105-15-172" -e "test22@ua.edu"| jq '.'

{
  "status": "ok",
  "message-type": "work",
  "message-version": "1.0.0",
  "message": {
    "indexed": {
      "date-parts": [
        [
          2023,
          6,
          8
        ]
      ],
      "date-time": "2023-06-08T12:12:12Z",
      "timestamp": 1686226332379
    },
    "reference-count": 20,
    "publisher": "Springer Science and Business Media LLC",
    "issue": "1",
    "content-domain": {
      "domain": [
        "link.springer.com"
      ],
      "crossmark-restriction": false
    },
    "short-container-title": [
      "BMC Bioinformatics"
    ],
    "published-print": {
      "date-parts": [
        [
          2014,
          12
        ]
      ]
    },
    "DOI": "10.1186/1471-2105-15-172",
    "type": "journal-article",
    "created": {
      "date-parts": [
        [
          2014,
          6,
          6
        ]
      ],
      "date-time": "2014-06-06T21:03:34Z",
      "timestamp": 1402088614000
    },
    "update-policy": "http://dx.doi.org/10.1007/spr

### Retrieve names from the file

In [15]:
!./crossref_api  | jq '.message.author[0].given'

"Peter"


In [16]:
%%bash
length=$(./crossref_api | jq '.["message"]["author"] | length')
for ((n = 0; n < $length; n++)); do
    first=$(./crossref_api  | jq ".message.author[$n].given" | tr -d '"')
    last=$(./crossref_api | jq ".message.author[$n].family" | tr -d '"')
    echo "$first $last";
done


Peter Ertl
Bernhard Rohde


### Save JSON data to a file
This is particularly useful for downstream testing or returning to results in the future (e.g., no need to keep requesting the data from crossref, save the results to a file)

In [17]:
!./crossref_api | jq '.' > my_data.json

## 2. Crossref API call with a Loop

In [18]:
!doi_List=('10.1021/acsomega.1c03250' '10.1021/acsomega.1c05512' '10.1021/acsomega.8b01647' '10.1021/acsomega.1c04287' '10.1021/acsomega.8b01834');\
for doi in "${doi_List[@]}"; \
do ./crossref_api -d "$doi" | jq --arg location "$doi" '.'; \
sleep 1 ; \
done

{
  "status": "ok",
  "message-type": "work",
  "message-version": "1.0.0",
  "message": {
    "indexed": {
      "date-parts": [
        [
          2023,
          7,
          20
        ]
      ],
      "date-time": "2023-07-20T17:50:34Z",
      "timestamp": 1689875434283
    },
    "reference-count": 45,
    "publisher": "American Chemical Society (ACS)",
    "issue": "36",
    "license": [
      {
        "start": {
          "date-parts": [
            [
              2021,
              9,
              1
            ]
          ],
          "date-time": "2021-09-01T00:00:00Z",
          "timestamp": 1630454400000
        },
        "content-version": "unspecified",
        "delay-in-days": 0,
        "URL": "https://creativecommons.org/licenses/by-nc-nd/4.0/"
      }
    ],
    "funder": [
      {
        "DOI": "10.13039/501100006261",
        "name": "Taif University",
        "doi-asserted-by": "publisher",
        "award": [
          "TURSP2020/68"
        ]
      },
    

In [19]:
!doi_List=('10.1021/acsomega.1c03250' '10.1021/acsomega.1c05512' '10.1021/acsomega.8b01647' '10.1021/acsomega.1c04287' '10.1021/acsomega.8b01834');\
for doi in "${doi_List[@]}"; \
do ./crossref_api -d "$doi" | jq --arg location "$doi" '.["message"]["title"][0]'; \
sleep 1 ; \
done

"Navigating into the Chemical Space of Monoamine Oxidase Inhibitors by Artificial Intelligence and Cheminformatics Approach"
"Impact of Artificial Intelligence on Compound Discovery, Design, and Synthesis"
"How Precise Are Our Quantitative Structure–Activity Relationship Derived Predictions for New Query Chemicals?"
"Applying Neuromorphic Computing Simulation in Band Gap Prediction and Chemical Reaction Classification"
"QSPR Modeling of the Refractive Index for Diverse Polymers Using 2D Descriptors"


In [20]:
!doi_List=('10.1021/acsomega.1c03250' '10.1021/acsomega.1c05512' '10.1021/acsomega.8b01647' '10.1021/acsomega.1c04287' '10.1021/acsomega.8b01834');\
for doi in "${doi_List[@]}"; \
do ./crossref_api -d "$doi" | jq --arg location "$doi" '.["message"]["author"][].affiliation[0].name'; \
sleep 1 ; \
done

"Department of Pharmaceutical Chemistry and Analysis, Amrita School of Pharmacy, Amrita Vishwa Vidyapeetham, AIMS Health Sciences Campus, Kochi 682041, India"
"Department of Pharmaceutical Chemistry and Analysis, Amrita School of Pharmacy, Amrita Vishwa Vidyapeetham, AIMS Health Sciences Campus, Kochi 682041, India"
"Department of Pharmaceutical Chemistry and Analysis, Amrita School of Pharmacy, Amrita Vishwa Vidyapeetham, AIMS Health Sciences Campus, Kochi 682041, India"
"Department of Pharmaceutical Chemistry and Analysis, Amrita School of Pharmacy, Amrita Vishwa Vidyapeetham, AIMS Health Sciences Campus, Kochi 682041, India"
"Department of Pharmaceutical Chemistry and Analysis, Amrita School of Pharmacy, Amrita Vishwa Vidyapeetham, AIMS Health Sciences Campus, Kochi 682041, India"
"Department of Pharmaceutics and Industrial Pharmacy, College of Pharmacy, Taif University, P.O. Box 11099, Taif 21944, Saudi Arabia"
"Department of Pharmaceutical Chemistry, College of Pharmacy, Jouf Univ

## 3. Crossref API call for journal information

We change the directory back to the CrossRef folder to create a new one for our project

In [ ]:
%cd ..

We set a directory where we want the the jornal_api program to be made

In [22]:
!mkdir journal_api

Then we change the directory to the folder we created

In [ ]:
%cd journal_api

Then we utilize the following command (%%file) to create the following makefile which will compile our program and create an executable.

In [ ]:
%%file makefile

#sets the variable CC to gcc, which is used to build the program
CC=gcc

#enable debugging information and enable all compiler warnings
CFLAGS=-g -Wall

#sets the bin variable as the name of the binary file we are creating
BIN=journal_api

#creates the binary file with the name we put
all: $(BIN)

#maps any file ending in .c to a binary executable. 
#"$<" represents the .c file and "$@" represents the target binary executable
%: %.c

#compiles the .c file using the gcc compiler with the CFLAGS and links 
#resulting binary with the CURL library
	$(CC) $(CFLAGS) $< -o $@ -lcurl

#clean target which removes specific files
clean:

#removes the binary file and an ".dSYM" (debug symbols for debugging) directories
#the RM command used -r to remove directories and -f to force delete
	$(RM) -rf $(BIN) *.dSYM


The command is used again to create our .c file which contains the code for the program

In [ ]:
%%file journal_api.c

#include <curl/curl.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

/*CURL program that retrieves the journal JSON data from the CrossRef API
This program allows custom request to be used along with the parameter*/


/* We are going to be inputting the custom email and issn with query like this: ./crossref_api -e "your_email@ua.edu" -i "1471-2105"
If the arguments are missing then we use the default: "your_email@ua.edu" "1471-2105"
*/

int main (int argc, char* argv[]){
    
    //if arguments are invalid just return
    if (argc > 5){                                                                                      
        printf("Error. Please try again correctly.\n");
        return -1;
    }

    //default parameter and request codes
    char parameter[100] = {};
    char request[500] = {}; 

    //if there is ./crossref_api -e/-i
    if ((argc == 1) || ((argc == 2) && ((strcmp(argv[1], "-e")==0) || (strcmp(argv[1], "-i")==0)))){
        //these arguments run the default parameters and keeps the codes as they are
        strcat(parameter,"your_email@ua.edu");
        strcat(request, "1471-2105");
    }

    //if there is ./crossref_api -e "your_email@ua.edu"
    else if ((argc == 3) && (strcmp(argv[1], "-e")==0)){
        //only the parameter code is changed
        strcat(parameter,argv[2]);
        strcat(request, "1471-2105");
    }

    //if there is ./crossref_api -e "your_email@ua.edu" -i
    else if ((argc == 4) && (strcmp(argv[1], "-e")==0) && (strcmp(argv[3], "-i")==0)){
        //only the parameter code is changed
        strcat(parameter,argv[2]);
        strcat(request, "1471-2105");
    }

    //if there is ./crossref_api -e "your_email@ua.edu" -i "1471-2105"
    else if ((argc == 5) && (strcmp(argv[1], "-e")==0) && (strcmp(argv[3], "-i")==0)){
        //both the parameter and request codes are changed
        strcat(parameter,argv[2]);
        strcat(request, argv[4]);
    }

    //if there is ./crossref_api -i "1471-2105"
    else if ((argc == 3) && (strcmp(argv[1], "-i")==0)){
        //only the request code is changed
        strcat(parameter,"your_email@ua.edu");
        strcat(request, argv[2]);
    }

    //if there is ./crossref_api -i "1471-2105" -e
    else if ((argc == 4) && (strcmp(argv[1], "-i")==0) && (strcmp(argv[3], "-e")==0)){
        //only the request code is changed
        strcat(parameter,"your_email@ua.edu");
        strcat(request, argv[2]);
    }

    //if there is ./crossref_api -i "1471-2105" -e "your_email@ua.edu" 
    else if ((argc == 5) && (strcmp(argv[1], "-i")==0) && (strcmp(argv[3], "-e")==0)){
        //both the request and parameter codes are changed
        strcat(parameter,argv[4]);
        strcat(request, argv[2]);
    }

    else{
        printf("usage: ./crossref_api [-i] request [-e] parameter\n\n");
        printf("the api_req_par program is used to retrieve json data from the CrossRef API\n\n");
        printf("optional arguments\n");
        printf("\t -i issn with query         optional custom issn code; default is '1471-2105'\n");
        printf("\t -e email                   optional custom email; default is 'your_email@ua.edu'\n");
        return -1;
    }

    //initializes the CURL HTTP connection
    CURL *curl = curl_easy_init();

    //bits of the url that are joined together later
    char api[] = "https://api.crossref.org/journals/";                                                                     
    char type1[] = "?mailto=";                        
    char url[2000];

    //checks if CURL initialization is a success or not
    if (!curl){                                                                                         
        fprintf(stderr, "init failed\n");
        return EXIT_FAILURE;
    }
        
    //combines all the bits to produce a functioning url
    sprintf(url, "%s%s%s%s", api, request, type1, parameter);                                             
                                          
    
    //sets the url to which the HTTP request will be sent to
    //first parameter is for the initialized curl HTTP request, second for the option to be set, and third for the value to be set
    curl_easy_setopt(curl, CURLOPT_URL, url);

    //if result is not retrieved then output error
    CURLcode result = curl_easy_perform(curl);

    //if result is not retrieved then output error
    if (result != CURLE_OK){                                                                            
        fprintf(stderr, "download problem: %s\n", curl_easy_strerror(result));
    }

    //deallocates memory for the CURL connection
    curl_easy_cleanup(curl);                                                                            
    return EXIT_SUCCESS;
}

In [ ]:
!make

In [27]:
!./journal_api | jq '.'

{
  "status": "ok",
  "message-type": "journal",
  "message-version": "1.0.0",
  "message": {
    "last-status-check-time": 1690246980331,
    "counts": {
      "current-dois": 1511,
      "backfile-dois": 10498,
      "total-dois": 12009
    },
    "breakdowns": {
      "dois-by-issued-year": [
        [
          2010,
          861
        ],
        [
          2019,
          762
        ],
        [
          2008,
          745
        ],
        [
          2009,
          729
        ],
        [
          2011,
          722
        ],
        [
          2006,
          633
        ],
        [
          2021,
          622
        ],
        [
          2014,
          619
        ],
        [
          2007,
          613
        ],
        [
          2012,
          609
        ],
        [
          2013,
          607
        ],
        [
          2015,
          603
        ],
        [
          2020,
          585
        ],
        [
          2017,
          585


## 4. Crossref API - Get article DOIs for a journal

### Request DOI data from crossref API

We change the directory back to the CrossRef folder to create a new one for our project

In [ ]:
%cd ..

We set a directory where we want the the jornal_api program to be made

In [29]:
!mkdir journal_doi_api

Then we change the directory to the folder we created

In [ ]:
%cd journal_doi_api

Then we utilize the following command (%%file) to create the following makefile which will compile our program and create an executable.

In [ ]:
%%file makefile

#sets the variable CC to gcc, which is used to build the program
CC=gcc

#enable debugging information and enable all compiler warnings
CFLAGS=-g -Wall

#sets the bin variable as the name of the binary file we are creating
BIN=journal_doi

#creates the binary file with the name we put
all: $(BIN)

#maps any file ending in .c to a binary executable. 
#"$<" represents the .c file and "$@" represents the target binary executable
%: %.c

#compiles the .c file using the gcc compiler with the CFLAGS and links 
#resulting binary with the CURL library
	$(CC) $(CFLAGS) $< -o $@ -lcurl

#clean target which removes specific files
clean:

#removes the binary file and an ".dSYM" (debug symbols for debugging) directories
#the RM command used -r to remove directories and -f to force delete
	$(RM) -rf $(BIN) *.dSYM


The command is used again to create our .c file which contains the code for the program

In [ ]:
%%file journal_doi.c

#include <curl/curl.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

/*CURL program that retrieves the journal JSON data from the CrossRef API
This program allows custom request to be used along with the parameter*/


/* We are going to be inputting the custom email and issn with query like this: ./crossref_api -e "your_email@ua.edu" -i "1471-2105"
If the arguments are missing then we use the default: "your_email@ua.edu" "1471-2105"
*/

int main (int argc, char* argv[]){
    
    //if arguments are invalid just return
    if (argc > 5){                                                                                      
        printf("Error. Please try again correctly.\n");
        return -1;
    }

    //default parameter and request codes
    char parameter[100] = {};
    char request[500] = {}; 

    //if there is ./crossref_api -e/-i
    if ((argc == 1) || ((argc == 2) && ((strcmp(argv[1], "-e")==0) || (strcmp(argv[1], "-i")==0)))){
        //these arguments run the default parameters and keeps the codes as they are
        strcat(parameter,"your_email@ua.edu");
        strcat(request, "1471-2105");
    }

    //if there is ./crossref_api -e "your_email@ua.edu"
    else if ((argc == 3) && (strcmp(argv[1], "-e")==0)){
        //only the parameter code is changed
        strcat(parameter,argv[2]);
        strcat(request, "1471-2105");
    }

    //if there is ./crossref_api -e "your_email@ua.edu" -i
    else if ((argc == 4) && (strcmp(argv[1], "-e")==0) && (strcmp(argv[3], "-i")==0)){
        //only the parameter code is changed
        strcat(parameter,argv[2]);
        strcat(request, "1471-2105");
    }

    //if there is ./crossref_api -e "your_email@ua.edu" -i "1471-2105"
    else if ((argc == 5) && (strcmp(argv[1], "-e")==0) && (strcmp(argv[3], "-i")==0)){
        //both the parameter and request codes are changed
        strcat(parameter,argv[2]);
        strcat(request, argv[4]);
    }

    //if there is ./crossref_api -i "1471-2105"
    else if ((argc == 3) && (strcmp(argv[1], "-i")==0)){
        //only the request code is changed
        strcat(parameter,"your_email@ua.edu");
        strcat(request, argv[2]);
    }

    //if there is ./crossref_api -i "1471-2105" -e
    else if ((argc == 4) && (strcmp(argv[1], "-i")==0) && (strcmp(argv[3], "-e")==0)){
        //only the request code is changed
        strcat(parameter,"your_email@ua.edu");
        strcat(request, argv[2]);
    }

    //if there is ./crossref_api -i "1471-2105" -e "your_email@ua.edu" 
    else if ((argc == 5) && (strcmp(argv[1], "-i")==0) && (strcmp(argv[3], "-e")==0)){
        //both the request and parameter codes are changed
        strcat(parameter,argv[4]);
        strcat(request, argv[2]);
    }

    else{
        printf("usage: ./crossref_api [-i] request [-e] parameter\n\n");
        printf("the api_req_par program is used to retrieve json data from the CrossRef API\n\n");
        printf("optional arguments\n");
        printf("\t -i issn with query         optional custom issn code; default is '1471-2105'\n");
        printf("\t -e email                   optional custom email; default is 'your_email@ua.edu'\n");
        return -1;
    }

    //initializes the CURL HTTP connection
    CURL *curl = curl_easy_init();

    //bits of the url that are joined together later
    char api[] = "https://api.crossref.org/journals/";                                                                     
    char type1[] = "&mailto=";                        
    char url[2000];

    //checks if CURL initialization is a success or not
    if (!curl){                                                                                         
        fprintf(stderr, "init failed\n");
        return EXIT_FAILURE;
    }
        
    //combines all the bits to produce a functioning url
    sprintf(url, "%s%s%s%s", api, request, type1, parameter);                                             
                                          
    
    //sets the url to which the HTTP request will be sent to
    //first parameter is for the initialized curl HTTP request, second for the option to be set, and third for the value to be set
    curl_easy_setopt(curl, CURLOPT_URL, url);

    //if result is not retrieved then output error
    CURLcode result = curl_easy_perform(curl);

    //if result is not retrieved then output error
    if (result != CURLE_OK){                                                                            
        fprintf(stderr, "download problem: %s\n", curl_easy_strerror(result));
    }

    //deallocates memory for the CURL connection
    curl_easy_cleanup(curl);                                                                            
    return EXIT_SUCCESS;
}

In [ ]:
!make

In [34]:
!./journal_doi -i "1471-2105/works?filter=from-pub-date:2014,until-pub-date:2014&select=DOI"| jq '.'

{
  "status": "ok",
  "message-type": "work-list",
  "message-version": "1.0.0",
  "message": {
    "facets": {},
    "total-results": 619,
    "items": [
      {
        "DOI": "10.1186/s12859-014-0411-1"
      },
      {
        "DOI": "10.1186/1471-2105-15-s10-p24"
      },
      {
        "DOI": "10.1186/1471-2105-15-s1-s2"
      },
      {
        "DOI": "10.1186/1471-2105-15-s10-p32"
      },
      {
        "DOI": "10.1186/1471-2105-15-s16-s13"
      },
      {
        "DOI": "10.1186/1471-2105-15-49"
      },
      {
        "DOI": "10.1186/1471-2105-15-s11-i1"
      },
      {
        "DOI": "10.1186/1471-2105-15-88"
      },
      {
        "DOI": "10.1186/1471-2105-15-331"
      },
      {
        "DOI": "10.1186/1471-2105-15-s11-s10"
      },
      {
        "DOI": "10.1186/1471-2105-15-s1-s8"
      },
      {
        "DOI": "10.1186/1471-2105-15-s14-s1"
      },
      {
        "DOI": "10.1186/1471-2105-15-s10-p33"
      },
      {
        "DOI": "10.1186/1471-2105-15-78"


By default, 20 results are displayed. Crossref allows up to 1000 returned results using the rows parameter. To get all 619 results, we can increase the number of returned rows.

We add the following to our query "&rows=700" to view all results

In [35]:
!./journal_doi -i "1471-2105/works?filter=from-pub-date:2014,until-pub-date:2014&select=DOI&rows=700"| jq '.' > dois_save.json

### Extract DOIs

In [36]:
!cat dois_save.json | jq '.["message"]["items"][].DOI'

"10.1186/s12859-014-0411-1"
"10.1186/1471-2105-15-s10-p24"
"10.1186/1471-2105-15-s1-s2"
"10.1186/1471-2105-15-s10-p32"
"10.1186/1471-2105-15-s16-s13"
"10.1186/1471-2105-15-49"
"10.1186/1471-2105-15-s11-i1"
"10.1186/1471-2105-15-88"
"10.1186/1471-2105-15-331"
"10.1186/1471-2105-15-s11-s10"
"10.1186/1471-2105-15-s1-s8"
"10.1186/1471-2105-15-s14-s1"
"10.1186/1471-2105-15-s10-p33"
"10.1186/1471-2105-15-78"
"10.1186/1471-2105-15-s10-p6"
"10.1186/1471-2105-15-115"
"10.1186/1471-2105-15-61"
"10.1186/1471-2105-15-24"
"10.1186/1471-2105-15-105"
"10.1186/s12859-014-0400-4"
"10.1186/1471-2105-15-106"
"10.1186/1471-2105-15-s7-s1"
"10.1186/s12859-014-0415-x"
"10.1186/1471-2105-15-s9-s12"
"10.1186/1471-2105-15-s13-s1"
"10.1186/1471-2105-15-73"
"10.1186/1471-2105-15-s10-p5"
"10.1186/1471-2105-15-122"
"10.1186/1471-2105-15-178"
"10.1186/1471-2105-15-s6-s3"
"10.1186/1471-2105-15-236"
"10.1186/1471-2105-15-318"
"10.1186/1471-2105-15-189"
"10.1186/1471-2105-15-199"
"10.1186/1471-2105-15-101"
"10.1186/147

In [37]:
!cat dois_save.json | jq '.["message"]["items"][].DOI' | wc -l

619



**What if we have more than 1000 results in a single query?**

For example, if we wanted the DOIs from *BMC Bioinformatics* for years 2014 through 2016?

In [38]:
!./journal_doi -i "1471-2105/works?filter=from-pub-date:2014,until-pub-date:2016&select=DOI"| jq '.'

{
  "status": "ok",
  "message-type": "work-list",
  "message-version": "1.0.0",
  "message": {
    "facets": {},
    "total-results": 1772,
    "items": [
      {
        "DOI": "10.1186/s12859-015-0538-8"
      },
      {
        "DOI": "10.1186/1471-2105-16-s18-s5"
      },
      {
        "DOI": "10.1186/s12859-015-0621-1"
      },
      {
        "DOI": "10.1186/s12859-015-0468-5"
      },
      {
        "DOI": "10.1186/s12859-015-0717-7"
      },
      {
        "DOI": "10.1186/s12859-014-0411-1"
      },
      {
        "DOI": "10.1186/1471-2105-16-s15-p13"
      },
      {
        "DOI": "10.1186/1471-2105-15-s10-p24"
      },
      {
        "DOI": "10.1186/s12859-016-1297-x"
      },
      {
        "DOI": "10.1186/s12859-015-0791-x"
      },
      {
        "DOI": "10.1186/1471-2105-15-s1-s2"
      },
      {
        "DOI": "10.1186/1471-2105-16-s10-s1"
      },
      {
        "DOI": "10.1186/s12859-015-0834-3"
      },
      {
        "DOI": "10.1186/1471-2105-15-s10-p32"

For example, if we wanted the DOIs from BMC Bioinformatics for years 2014 through 2016, we see that there are 1772 DOIs:

In [39]:
!./journal_doi -i "1471-2105/works?filter=from-pub-date:2014,until-pub-date:2016&select=DOI"| jq '.["message"]["total-results"]'

1772


Here we see that the total results is over 1000 (total-results: 1772). An additional parameter that we can use with crossref API is called "offset". The offset option allows us to select sets of records and define a starting position (e.g., the first 1000, and then the second set of up to 1000.)

We add the %%bash command run the whole loop

In [40]:
%%bash
length=$(./journal_doi -i "1471-2105/works?filter=from-pub-date:2014,until-pub-date:2016&select=DOI" | jq '.["message"]["total-results"]')
for ((n = 0; n < $length; n += 1000)); do
    ./journal_doi -i "1471-2105/works?filter=from-pub-date:2014,until-pub-date:2016&select=DOI&rows=1000&offset=$n" \
    | jq '.["message"]["items"][].DOI' \
    >> dois_save2.txt
    sleep 1
done


In [41]:
!head dois_save2.txt

"10.1186/s12859-015-0538-8"
"10.1186/1471-2105-16-s18-s5"
"10.1186/s12859-015-0621-1"
"10.1186/s12859-015-0468-5"
"10.1186/s12859-015-0717-7"
"10.1186/s12859-016-0985-x"
"10.1186/1471-2105-15-172"
"10.1186/s12859-014-0411-1"
"10.1186/1471-2105-16-s15-p13"
"10.1186/1471-2105-15-s10-p24"


In [42]:
!cat dois_save2.txt | wc -l

1772
